## Lineaire regressie
Lineaire regressie is een centrale techniek binnen Machine Learning. Om de wiskunde van ML te begrijpen is het een absolute must om deze techniek ten volle te doorgronden.  
  
### Data simulatie
We beginnen met de simulatie van een simpel patroon:

